In [15]:
new_path = r'datasets\\1\\RecipeNLG_dataset.csv'

In [16]:
import pandas as pd
df = pd.read_csv(new_path)
df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [ ]:
import ast
df["NER"] = df["NER"].apply(ast.literal_eval)

In [13]:
ingredients = set(i for row in df["NER"] for i in row)
print(f"Number of all ingredients: {len(ingredients)}")

Number of all ingredients: 234059


In [14]:
len(df)

2231142

In [19]:
from sentence_transformers import SentenceTransformer
import torch

# GPU for recipe embeddings
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
recipe_embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)

# CPU for user preference embeddings
user_embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cpu')


cuda


In [20]:
recipe_texts = df['NER'].apply(lambda x: " ".join(x)).tolist()
recipe_embeddings = recipe_embedder.encode(recipe_texts, convert_to_tensor=True, device=device)

In [21]:
import numpy as np

# recipe_embeddings is a PyTorch tensor on GPU
recipe_embeddings_np = recipe_embeddings.cpu().numpy().astype('float32')


In [23]:
import faiss
embedding_dim = recipe_embeddings_np.shape[1]  # e.g., 384
index = faiss.IndexFlatL2(embedding_dim)       # L2 distance metric
index.add(recipe_embeddings_np)                # add recipe vectors
print(f"Number of vectors in index: {index.ntotal}")
faiss.write_index(index, "faiss_indexes/faiss_recipe_index.index")
print("FAISS index saved successfully!")

Number of vectors in index: 2231142
FAISS index saved successfully!
